In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl python-dotenv
!pip list |egrep 'accelerate|peft|bitsandbytes|transformers|trl'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
accelerate                         0.34.2
bitsandbytes                       0.44.1
fastrlock                          0.8.2
peft                               0.13.2
transformers                       4.44.2
trl                                0.11.4


In [2]:
from huggingface_hub import login
import os
load_dotenv()
access_token_read = os.getenv("HUGGINGFACE_TOKEN")
login(token = access_token_read)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format
model_name="meta-llama/Llama-3.2-3B-Instruct"
peft_model = "youzarsif/Doctor_Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(peft_model)

base_model_reload= AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [4]:
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)
model = PeftModel.from_pretrained(base_model_reload, peft_model)

model = model.merge_and_unload()

adapter_config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.65G [00:00<?, ?B/s]

In [9]:
from pprint import pprint
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

instruction = """
You are an AI trained to provide accurate health information and interpret scientific research. 
Your goal is to answer medical questions, suggest treatments, offer general health advice based on established guidelines, 
and provide evidence-based insights by analyzing relevant research studies. When necessary, summarize clinical findings to support your advice.
"""
query = """
My wife was admitted to hospital for sever pain in stomach. Ultrasound showed 4-5 polys in Galbladder of 3 to 4 mm each.
As per doctor it gasto intestine pain and she was treated for same only. What is treatment of polys? Is it something to worry?
Can it be treated with medeicenes ? What is the treatment for polyps in gall bladder?
"""

messages = [{"role": "system", "content": instruction},
    {"role": "user", "content": query}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to(device)

outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1, temperature=0.7, top_k=50, top_p=0.9,repetition_penalty=1.2 )

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

pprint(text.split("assistant")[1])

('\n'
 'Hi I have gone through your question regarding ultrasound finding of a '
 'patient who has severe abdominal pain (appendicitis) and after getting all '
 'reports done as follows: Polys or more common called polyp (polyps means '
 'small growths), which appear to grow from wall lining, may cause problems if '
 'they bleed into adjacent structures or become infected. The most important '
 'thing here is that there were no signs of bleeding into surrounding tissues '
 '(peritoneal fluid). It appears like infection might have caused them (due to '
 'high fever temperature body gets when there is any infection in the body). '
 'Infection can occur due to bacteria infecting these growths or even stones '
 'blocking bile ducts causing obstruction leading to bacterial infection '
 'within these obstructed areas. If not already')
